# Punjab MLA Srapping

### Extraction from details screen

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Set up Selenium with Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless (without GUI)

# Initialize the WebDriver
driver = webdriver.Chrome('../chrome_driver/chromedriver.exe')

# URL to scrape
url = "https://www.punjabassembly.nic.in/Members/MembersProfile?MemberID=4&AssemblyID=16"

# Visit the page
driver.get(url)

# Wait for the page to load and click the 'Proceed' button (example)
driver.find_element(By.ID, "details-button").click()  # Adjust based on actual element ID

# Get the page source after interaction
driver.find_element(By.ID, 'proceed-link').click()


# Get the page source after interaction
soup = BeautifulSoup(driver.page_source, "html.parser")


# Extract the required details
# Extract Name
name_tag = soup.find("span", string="Sh. Aman Arora")  # The exact name to search for
name = name_tag.text.strip() if name_tag else "Not Found"

# Extract State Name
state_tag = soup.find("span", string="Punjab")  # The exact state to search for
state = state_tag.text.strip() if state_tag else "Not Found"

# Extract Constituency Name
constituency_tag = soup.find("p", class_="text-wrap")
constituency = constituency_tag.text.strip() if constituency_tag else ""

# Extract Mobile Number
phone_tag = soup.find("p", class_="text-wrap").find("a")
contact = phone_tag.text.strip() if phone_tag else ""

# Extract Email
email_tag = soup.find("p", class_="text-wrap").find("a")
email = email_tag.text.strip() if email_tag else ""

# Extract Party Name
party_tag = soup.find("h6", class_="text-wrap")
party_name = party_tag.text.strip() if party_tag else "Not Found"

# Gender and email are not available on the page based on this example
gender = ""

# Output the extracted details
print({
    "Constituency": constituency,
    "Name": name,
    "Gender": gender,
    "Party": party_name,
    "Contact": contact,
    "Email": email,
})

# Close the browser
driver.quit()


TypeError: find() got multiple values for argument 'name'

In [94]:
detail_page_links = []
data = []

In [100]:
def scrap_through_details(driver):
    
    for url in detail_page_links:

        driver.implicitly_wait(2)
        driver.get('https://www.punjabassembly.nic.in' + url)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        name_tag = soup.find("span", string="Sh. Aman Arora")  # The exact name to search for
        name = name_tag.text.strip() if name_tag else "Not Found"

        # Extract State Name
        state_tag = soup.find("span", string="Punjab")  # The exact state to search for
        state = state_tag.text.strip() if state_tag else "Not Found"

        # Extract Constituency Name
        constituency_tag = soup.find("p", class_="text-wrap")
        constituency = constituency_tag.text.strip() if constituency_tag else ""

        # Extract Mobile Number
        if (soup.find("p", class_="text-wrap") != None):
            phone_tag = soup.find("p", class_="text-wrap").find("a")
            contact = phone_tag.text.strip() if phone_tag else ""
        else:
            contact = ""

        # Extract Email
        if (soup.find("p", class_="text-wrap") != None):
            email_tag = soup.find("p", class_="text-wrap").find("a")
            email = email_tag.text.strip() if email_tag else ""
        else:
            email = ""

        # Extract Party Name
        party_tag = soup.find("h6", class_="text-wrap")
        party_name = party_tag.text.strip() if party_tag else "Not Found"

        # Gender and email are not available on the page based on this example
        gender = ""

        # Output the extracted details
        data.append({
            "Constituency": constituency,
            "Name": name,
            "Gender": gender,
            "Party": party_name,
            "Contact": contact,
            "Email": email,
        })


### Extracting Links from Grid

In [ ]:
import pandas as pd

# Set up Selenium with Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless (without GUI)
driver = webdriver.Chrome('../chrome_driver/chromedriver.exe')

for x in range(1, 6):

    url = f"https://www.punjabassembly.nic.in/Members?Page_No={x}"
    driver.get(url)

    if x == 1:
        # Processing the non-trusted site
        driver.find_element(By.ID, "details-button").click()
        driver.find_element(By.ID, 'proceed-link').click()


    # Starting Extraction
    soup = BeautifulSoup(driver.page_source, "html.parser")

    view_more_tags = soup.find_all("a", href=True, string="View More")
    view_more_links = [tag['href'] for tag in view_more_tags]

    # Append the Links
    detail_page_links.extend(view_more_links)

    # Output the extracted links
    print("View More Links:")
    for link in view_more_links:
        print('https://www.punjabassembly.nic.in' + link)



In [103]:
# Scrap Detail Pages
scrap_through_details(driver)

# Quit Driver
driver.quit()

# Save the Data
# Save data to CSV
df = pd.DataFrame(data)
df.to_csv('pubjab_mla_contact_details.csv', index=False)
print("Data saved to pubjab_mla_contact_details.csv")

Data saved to pubjab_mla_contact_details.csv
